In [7]:
#Adaptado: https://github.com/andersonrocha0/tpa
import nest_asyncio
nest_asyncio.apply()

import asyncio
import concurrent.futures
import sys
import time
from enum import Enum
#from multiprocessing import Pool, cpu_count

import aiohttp
import requests


def get_ids_cervejarias():
    r = requests.get('https://api.openbrewerydb.org/breweries')
    if r.status_code == 200:
        return [r['id'] for r in r.json()]


def get_cervejaria(id_c):
    r = requests.get(f'https://api.openbrewerydb.org/breweries/{id_c}')
    if r.status_code == 200:
        print(r.json())


async def get_cervejaria_async(id_c, session):
    async with session.get(f'https://api.openbrewerydb.org/breweries/{id_c}') as response:
        print(await response.json())


async def processar_async(ids):
    coros = []
    async with aiohttp.ClientSession() as session:
        for id_c in ids:
            coros.append(get_cervejaria_async(id_c, session))

        await asyncio.gather(*coros)


async def get_cervejaria_async_manual(id_c):
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(None, get_cervejaria, id_c)


class TipoExecucao(Enum):
    UM_PROCESSO = 1
    VARIAS_THREADS = 2
    ASYNCIO_COM_LIB_HTTP = 3

if __name__ == '__main__':
    started = time.time()

    tipo_execucao = TipoExecucao.VARIAS_THREADS
    cores = cpu_count()

    ids_cervejarias = get_ids_cervejarias()

    if TipoExecucao.UM_PROCESSO == tipo_execucao:
        # Usando somente um processo
        for id_cervejaria in ids_cervejarias:
            get_cervejaria(id_cervejaria)
            
            
    elif TipoExecucao.VARIAS_THREADS == tipo_execucao:
        # Usando várias threads
        with concurrent.futures.ThreadPoolExecutor(cores) as thp:
            thp.map(get_cervejaria, ids_cervejarias)
            
    elif TipoExecucao.ASYNCIO_COM_LIB_HTTP == tipo_execucao:
        # Usando asyncio
        event_loop = asyncio.get_event_loop()
        tasks = []

        event_loop.run_until_complete(processar_async(ids_cervejarias))
    elif TipoExecucao.ASYNCIO_MANUAL == tipo_execucao:
        # Usando asyncio manual
        event_loop = asyncio.get_event_loop()
        tasks = []

        for id_cervejaria in ids_cervejarias:
            tasks.append(get_cervejaria_async_manual(id_cervejaria))

        event_loop.run_until_complete(asyncio.wait(tasks))

    elapsed = time.time()
    print('Time taken :', elapsed - started)

    #sys.exit(0)

{'id': 'boring-brewing-co-llc-boring', 'name': 'Boring Brewing Co., LLC', 'brewery_type': 'micro', 'street': '29300 SE Haley Road Ste B', 'address_2': None, 'address_3': None, 'city': 'Boring', 'state': 'Oregon', 'county_province': None, 'postal_code': '97009', 'country': 'United States', 'longitude': None, 'latitude': None, 'phone': '5034278619', 'website_url': 'http://www.boringbrewing.com', 'updated_at': '2021-10-23T02:24:55.243Z', 'created_at': '2021-10-23T02:24:55.243Z'}
{'id': 'cape-ann-lanes-gloucester', 'name': 'Cape Ann Lanes', 'brewery_type': 'planning', 'street': None, 'address_2': None, 'address_3': None, 'city': 'Gloucester', 'state': 'Massachusetts', 'county_province': None, 'postal_code': '01930-2256', 'country': 'United States', 'longitude': None, 'latitude': None, 'phone': '9788799714', 'website_url': None, 'updated_at': '2021-10-23T02:24:55.243Z', 'created_at': '2021-10-23T02:24:55.243Z'}
{'id': 'dented-face-brewing-company-delta', 'name': 'Dented Face Brewing Company